# Site Machakos
#### **PostgreSQL**

Ensure that the site / machine already has the `nitras` database. If not, use the `CREATE DATABASE nitras;` SQL query 

## Tables

|Table|Description|
|---|---|
|**inventory_machakos**| Fragment of the inventory table |
| **category** | duplicate of the category table. Required since inventory_machakos references it as a foreign key |
|**suppliers** | duplicate of the suppliers table. Required since inventory_machakos references it as a foreign key |


### `category` Table

In [ ]:
-- Active: 1706439060085@@127.0.0.1@5432@nitras

CREATE TABLE category (
    id INT PRIMARY KEY,
    name VARCHAR(255) NOT NULL,
    storehouse VARCHAR(50) NOT NULL
);

INSERT INTO category (id, name, storehouse) VALUES
(1, 'Clothing', 'Nairobi'),
(2, 'Electronics', 'Nakuru'),
(3, 'Books', 'Machakos'),
(4, 'Toys', 'Nairobi'),
(5, 'Furniture', 'Nakuru');

### `suppliers` Table

In [ ]:
-- Active: 1706439060085@@127.0.0.1@5432@nitras

CREATE TABLE suppliers (
    id INT PRIMARY KEY,
    name VARCHAR(255) NOT NULL,
    email VARCHAR(255) NOT NULL,
    phone VARCHAR(20),
    address TEXT,
    products_count INT
);

INSERT INTO suppliers (id, name, email, phone, address, products_count) VALUES
(1, 'Supplier A', 'supplierA@example.com', '111-222-3333', '400 Supplier St', 5),
(2, 'Supplier B', 'supplierB@example.com', '444-555-6666', '500 Supplier St', 3),
(3, 'Supplier C', 'supplierC@example.com', '777-888-9999', '600 Supplier St', 7),
(4, 'Supplier D', 'supplierD@example.com', '999-888-7777', '700 Supplier St', 10),
(5, 'Supplier E', 'supplierE@example.com', '333-555-1111', '800 Supplier St', 8);

### `inventory_machakos` Table

In [ ]:
-- Active: 1706439060085@@127.0.0.1@5432@nitras

--
-- Create Global Table `inventory`
--
CREATE TABLE inventory (
    id INT PRIMARY KEY,
    name VARCHAR(255) NOT NULL,
    category_id INT,
    description TEXT,
    price DECIMAL(10, 2) NOT NULL,
    quantity INT NOT NULL,
    supplier_id INT,
    FOREIGN KEY (category_id) REFERENCES category(id),
    FOREIGN KEY (supplier_id) REFERENCES suppliers(id)
);

INSERT INTO inventory (id, name, category_id, description, price, quantity, supplier_id) VALUES
(1, 'T-Shirt', 1, 'Cotton T-Shirt', 19.99, 100, 1),
(2, 'Laptop', 2, 'High-performance laptop', 899.99, 30, 2),
(3, 'Harry Potter Book', 3, 'Fantasy novel', 15.99, 50, 3),
(4, 'Action Figure', 4, 'Superhero action figure', 9.99, 80, 4),
(5, 'Sofa', 5, 'Comfortable sofa', 499.99, 10, 5),
(6, 'Jeans', 1, 'Blue jeans', 29.99, 50, 1),
(7, 'Smartphone', 2, 'Latest smartphone model', 699.99, 20, 2),
(8, 'Cookbook', 3, 'Collection of recipes', 12.99, 40, 3),
(9, 'Board Game', 4, 'Family board game', 24.99, 60, 4),
(10, 'Coffee Table', 5, 'Wooden coffee table', 129.99, 15, 5),
(11, 'Dress', 1, 'Elegant dress', 39.99, 30, 1),
(12, 'Headphones', 2, 'Noise-canceling headphones', 149.99, 25, 2),
(13, 'Mystery Novel', 3, 'Thriller mystery novel', 18.99, 35, 3),
(14, 'Building Blocks', 4, 'Educational building blocks', 14.99, 70, 4),
(15, 'Bed Frame', 5, 'Queen-sized bed frame', 299.99, 8, 5),
(16, 'Polo Shirt', 1, 'Casual polo shirt', 24.99, 40, 1),
(17, 'Tablet', 2, 'Android tablet', 199.99, 15, 2),
(18, 'Science Book', 3, 'Scientific exploration book', 20.99, 25, 3),
(19, 'Puzzle Set', 4, 'Challenging puzzle set', 17.99, 55, 4),
(20, 'Bookshelf', 5, 'Wooden bookshelf', 179.99, 12, 5);

--
-- Create Fragment Table `inventory_machakos`
--
CREATE TABLE inventory_machakos AS
SELECT *
FROM inventory
WHERE category_id IN (SELECT id FROM category WHERE storehouse = 'Machakos');

INSERT INTO inventory_machakos
SELECT *
FROM inventory
WHERE category_id IN (SELECT id FROM category WHERE storehouse = 'Machakos');

--
-- Drop Global Table `inventory` as we just want this site to have the fragment
--
DROP TABLE inventory;


## Link Sites

### Create Server Connections

Since Machakos is the decision site, where all the reconstruction will be happening, the postgres server needs to be aware of all the sites.

In [ ]:
-- Active: 1706439060085@@127.0.0.1@5432@nitras

--  sudo apt-get install postgresql-<version>-mysql-fdw
-- https://github.com/EnterpriseDB/mysql_fdw
-- Uncomment the below query to load extension if installed for the first time. Otherwise no need to run it again.

CREATE EXTENSION mysql_fdw;

#### Prepare server connection to Nairobi site

In [ ]:
-- Active: 1706439060085@@127.0.0.1@5432@nitras

--
-- create MYSQL server object
--
CREATE SERVER mysql_server
	FOREIGN DATA WRAPPER mysql_fdw
	OPTIONS (host '24.144.94.53', port '3306');

--
-- create user mapping
--
CREATE USER MAPPING FOR admin451
	SERVER mysql_server
	OPTIONS (username 'admin451', password 'admin451');

#### Prepare server connection to Nakuru site

In [ ]:
-- Active: 1706439060085@@127.0.0.1@5432@nitras

--
-- create MariaDB server object
--
CREATE SERVER mariadb_server
	FOREIGN DATA WRAPPER mysql_fdw
	OPTIONS (host '146.190.146.117', port '3306');

--
-- create user mapping
--
CREATE USER MAPPING FOR admin451
	SERVER mariadb_server
	OPTIONS (username 'admin451', password 'admin451');

### Add Foreign Tables

The fragmented tables in the other sites need to be listed in PostgreSQL's `foreign_tables` relation which is in the `information_schema` database.

#### Add `inventory_nairobi` foreign table

In [ ]:
-- Active: 1706439060085@@127.0.0.1@5432@nitras

CREATE FOREIGN TABLE inventory_nairobi
	(
		id INT,
        name VARCHAR(255) NOT NULL,
        category_id INT,
        description TEXT,
        price DECIMAL(10, 2) NOT NULL,
        quantity INT NOT NULL,
        supplier_id INT
	)
	SERVER mysql_server
	OPTIONS (dbname 'nitras', table_name 'inventory_nairobi');

SELECT * FROM inventory_nairobi;


#### Add `inventory_nakuru` foreign table

In [ ]:
-- Active: 1706439060085@@127.0.0.1@5432@nitras

CREATE FOREIGN TABLE inventory_nakuru
	(
		id INT,
        name VARCHAR(255) NOT NULL,
        category_id INT,
        description TEXT,
        price DECIMAL(10, 2) NOT NULL,
        quantity INT NOT NULL,
        supplier_id INT
	)
	SERVER mariadb_server
	OPTIONS (dbname 'nitras', table_name 'inventory_nakuru');

SELECT * FROM inventory_nakuru;

You can check to see if the `foreign_tables` relation has been updated.

In [ ]:
-- Active: 1706439060085@@127.0.0.1@5432@nitras

select * from information_schema.foreign_tables;